In [4]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import datetime

# load data

In [7]:
data_dir = './data/tnic3_data/'
tmp_data_path = './data/tmp'


s_year = 1997
e_year = 2020

top_k = 10


In [8]:
tnic = pd.read_csv(data_dir+'tnic3_data.txt', sep='\t')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

tnic gvkey , from int to str

In [ ]:
tnic['gvkey1'], tnic['gvkey2'] = tnic['gvkey1'].astype(str), tnic['gvkey2'].astype(str)

In [ ]:
if type(tnic['gvkey1'][0]) == str:
    tnic.to_pickle(f'{tmp_data_path}/tnic_str.pickle')

In [ ]:
assert type(tnic.loc[1, 'gvkey1']) == str# check

In [9]:
df_ma = pd.read_pickle(f'{tmp_data_path}/ma_fv_{s_year}_{e_year}.pickle')

## Create Top-peer TNIC looking table



In [ ]:

# roughly 1min per year

tnic_top_k = tnic[(tnic.year >= 1997)].groupby(['year', 'gvkey1']).apply(lambda x: x.nlargest(top_k, columns=['score']))

tnic_top_k.to_pickle(f'{tmp_data_path}/tnic_top_{top_k}.pickle')

In [10]:
tnic_top_k = pd.read_pickle(f'{tmp_data_path}/tnic_top_{top_k}.pickle')

# the GVKEY we consider in TNIC

see [Appendix_x](Appendix_x_Q%26As.ipynb)
- they must be in top-k table (from 1997 to 2019) and met the requirement of financial variables




dimensions of Adjacency:

- refer to Dyrep: "Github dataset forms a large network with low clustering coefficient and sparse events thus allowing us to test the robustness of our model. Further, Github dataset contains several unseen nodes which were never encountered during training."

In [74]:
def get_unique(lst1, lst2):
    uniq = list(set(lst1+lst2))
    print('the length of unique elements is:', len(uniq))
    return uniq

In [11]:
tnic_topk_fv = pd.read_pickle(f'{tmp_data_path}/tnic_topk_finvar_{s_year}_2019.pickle')

In [19]:
gvkey_lst_tnic = list(tnic_topk_fv['gvkey'].unique())

Only keep the gvkey which are able to find enough financial information from Compustat

Why we need 2 mapping tables?

- In Dyrep, Only node in the network (in the adjacency matrix) could be involved in an event.
- In MA, even the node outside the network could be involved in an event as long as it has sufficiant financial variables 


So:

- the `gvkey_index_map`(larger one) contains `~_tnic`.


`~_tnic`(smaller one)

- use for construct Adjacency matrix

In [20]:
gvkey_index_map_tnic = {}
for i, gvkey in enumerate(gvkey_lst_tnic):
    gvkey_index_map_tnic[gvkey] = i # integer

In [21]:
index_gvkey_map_tnic = {}
for i in range(len(gvkey_lst_tnic)):
    index_gvkey_map_tnic[i] = gvkey_lst_tnic[i]    

In [53]:
len(gvkey_lst_tnic)

13494

`gvkey_index_map` (the larger one)

- use for ultimatly identify a firm 
- retreive financial varibales 

In [26]:

ma_gvkey_lst_unique = list(set(list(df_ma['AGVKEY'].unique()) + list(df_ma['TGVKEY'].unique())))


In [27]:
ultimate_gvkey_lst_unique = list(set(ma_gvkey_lst_unique + gvkey_lst_tnic))

In [28]:
assert len(ultimate_gvkey_lst_unique) > len(gvkey_lst_tnic)

In [32]:
print(' num of GVKEYS in ma which is not in the network:', len(ultimate_gvkey_lst_unique) - len(gvkey_lst_tnic))

 num of GVKEYS in ma which is not in the network: 238


should we remove the MA events which contain the firm that was not a node in the network? 

-- yes, make it simple

In [ ]:
# gvkey_index_map = {}
# for i, gvkey in enumerate(ultimate_gvkey_lst_unique):
#     gvkey_index_map_tnic[gvkey] = i # integer

In [ ]:
# gvkey_index_map_tnic = {}
# for i, gvkey in enumerate(gvkey_lst_tnic):
#     gvkey_index_map_tnic[gvkey] = i # integer

# Prepare Data for Dataloader

the key is the adjacency matrix

## transform ma data 


the `all_events` attribute should be a list containning element like(a 4 elements tuple):

```
(74, 6, 'PushEvent', datetime.datetime(2013, 1, 1, 3, 53, 4))

```


In [44]:
def convert_df(df):
    all_events = []
    not_found = 0
    for _, row in df_ma.iterrows():
        try:
            tuple_ = (gvkey_index_map_tnic[row.AGVKEY], gvkey_index_map_tnic[row.TGVKEY], 'ma', row.DA.to_pydatetime())  
            all_events.append(tuple_)
        except:
            not_found+=1 # not found 
            #print(row.AGVKEY, 'or', row.TGVKEY, 'not found in map')
        
    print(f'{not_found} MA event not in the network')
    return all_events
    

In [45]:
all_events = convert_df(df_ma)

1113 MA event not in the network


In [46]:
len(all_events)

5932

# prepare variables


gvkey_ids, A_initial, A_last, ma_events

`gvkey_ids`: a dict, gvkey_ids[gvkey] = id

In [49]:
gvkey_ids = gvkey_index_map_tnic

In [50]:
N_nodes = len(gvkey_ids)

In [52]:
A_initial = np.zeros((N_nodes, N_nodes))

In [54]:
dim = 13000
start_time = time.time()
a,b = np.random.random((dim, dim)), np.random.random((dim, dim)) 
c = a.dot(b)
print(c.shape)
print("--- %s seconds ---" % (time.time() - start_time))

(13000, 13000)
--- 20.66449475288391 seconds ---


for example, 1997